In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df =  pd.read_csv('../input/train.csv', nrows = 10_000_000)

In [ ]:
train_df.info()

In [ ]:
train_df.head()

In [ ]:
def clean_df(df):
    return df[(df.fare_amount > 0) & 
            (df.pickup_longitude > -80) & (df.pickup_longitude < -70) &
            (df.pickup_latitude > 35) & (df.pickup_latitude < 45) &
            (df.dropoff_longitude > -80) & (df.dropoff_longitude < -70) &
            (df.dropoff_latitude > 35) & (df.dropoff_latitude < 45) &
            (df.passenger_count > 0) & (df.passenger_count < 10)]

train_df = clean_df(train_df)

In [ ]:
def havesine_np(lon1 , lat1 , lon2 , lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [ ]:
dist = havesine_np(train_df['pickup_longitude'],train_df['pickup_latitude'],train_df['dropoff_longitude'],train_df['dropoff_latitude'])

In [ ]:
train_df['dist'] = dist

In [ ]:
train_df.head()

In [ ]:
print(train_df.isnull().sum())

In [ ]:
train_df = train_df.dropna(how = 'any', axis = 'rows')

In [ ]:
print(train_df.isnull().sum())

In [ ]:
train_df.drop(columns=['key', 'pickup_datetime'], inplace=True)
train_df.head()

In [ ]:
y = train_df['fare_amount']
X = train_df.drop(columns=['fare_amount'])

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.head()

In [ ]:
import xgboost as xgb

In [ ]:
params = {
    
    'max_depth': 8, 
    'eta':.03,
    'subsample': 1, 
    'colsample_bytree': 0.8, 
   
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 1
}

In [ ]:
xgdmat=xgb.DMatrix(X_train,y_train)
final_gb=xgb.train(params,xgdmat)


In [ ]:
test_df =  pd.read_csv('../input/test.csv', nrows = 10_000_000)

In [ ]:
test_df.info()

In [ ]:
dist = havesine_np(test_df['pickup_longitude'],test_df['pickup_latitude'],test_df['dropoff_longitude'],test_df['dropoff_latitude'])

In [ ]:
test_df['dist'] = dist

In [ ]:
print(test_df.isnull().sum())

In [ ]:
test_df.drop(columns=['key', 'pickup_datetime'], inplace=True)
test_df.head()

In [ ]:
tesdmat=xgb.DMatrix(test_df)
y_pred=final_gb.predict(tesdmat)

In [ ]:
a = pd.read_csv('../input/test.csv')

In [ ]:
submission = pd.DataFrame({
        "key": a.key,
        "fare_amount": y_pred
})

submission.to_csv('taxi_fare_submission.csv',index=False)
submission.head()